In [ ]:
import pandas as pd

def update_group_stage_points(gs_df, teams_df, season):
    for _, match in gs_df.iterrows():
        home_score, away_score = map(int, match['Result'].split(' - '))
        home_team, away_team = season + match['Home Team'], season + match['Away Team']
        if home_score > away_score:
            teams_df.loc[teams_df['Team'] == home_team, 'Group Stage Point'] += 3
        elif away_score > home_score:
            teams_df.loc[teams_df['Team'] == away_team, 'Group Stage Point'] += 3
        else:
            teams_df.loc[teams_df['Team'].isin([home_team, away_team]), 'Group Stage Point'] += 1

def update_highest_round(tourn_df, teams_df, season):
    round_map = {'R16': '16', 'QF': '8', 'SF': '4', 'Final': '2'}
    for _, match in tourn_df.iterrows():
        round_key = match['Round Number'].split()[0]
        if round_key in round_map:
            teams_df.loc[teams_df['Team'].isin([season + match['Home Team'], season + match['Away Team']]), 'Highest Round'] = round_map[round_key]

def update_champion(df, teams_df, season):
    final_match = df.iloc[-1]
    home_goals, away_goals = map(int, final_match['Result'].split(' - '))
    winner = final_match['Home Team'] if home_goals > away_goals else final_match['Away Team']
    teams_df.loc[teams_df['Team'] == season + winner, 'Champion'] = 1

def load_and_prepare_data(season, ucl_url, domestic_url):
    ucl_df = pd.read_csv(ucl_url)
    domestic_df = pd.read_csv(domestic_url)
    domestic_df['Team'] = season + domestic_df['Team'].astype(str)
    return ucl_df, domestic_df

def process_season(season, ucl_url, domestic_url):
    ucl_df, domestic_df = load_and_prepare_data(season, ucl_url, domestic_url)
    gs_df, tourn_df = ucl_df[ucl_df['Group'].notna()], ucl_df[ucl_df['Group'].isna()]
    list_of_teams = gs_df['Home Team'].unique()
    list_of_teams.sort()
    team_df = pd.DataFrame({'Team': season + list_of_teams, 'Group Stage Point': 0, 'Highest Round': '0', 'Champion': 0})
    update_group_stage_points(gs_df, team_df, season)
    update_highest_round(tourn_df, team_df, season)
    team_df = pd.merge(team_df, domestic_df, on='Team', how='left')
    update_champion(ucl_df, team_df, season)
    return team_df

seasons = ['1718', '1819', '1920', '2021', '2122', '2223']
final_df = pd.DataFrame()

for season in seasons:
    UCL_URL = 'https://raw.githubusercontent.com/DKunLee/DS2500_Final_Data/main/champions-league-' + season + '.csv'
    UCL_Domestic_URL = 'https://raw.githubusercontent.com/DKunLee/DS2500_Final_Data/main/UCL_GS_DomesticLeague_' + season + '.csv'

    season_df = process_season(season, UCL_URL, UCL_Domestic_URL)
    final_df = pd.concat([final_df, season_df], ignore_index=True)

cols = [col for col in final_df.columns if col != 'Champion'] + ['Champion']
final_df = final_df[cols]

final_df = final_df.astype({col: float for col in ['Group Stage Point', 'Highest Round', 'Played', 'Goal Difference', 'Points', 'League Point', 'Champion']})
final_df['Goal Difference'] = (final_df['Goal Difference'] / final_df['Played']).round(2)
final_df['Points'] = (final_df['Points'] / final_df['Played']).round(2)

final_df.describe()

,Group Stage Point,Highest Round,Played,Goal Difference,Points,League Point,Champion
count,192.000000,192.000000,192.000000,192.000000,192.000000,192.000000,192.000000
mean,8.359375,5.375000,17.140625,1.153177,2.089635,63.470208,0.031250
std,4.699450,6.677696,2.503613,0.751938,0.437292,26.936070,0.174448
min,0.000000,0.000000,11.000000,-0.720000,0.720000,11.250000,0.000000
25%,4.000000,0.000000,15.000000,0.575000,1.775000,37.900000,0.000000
50%,8.000000,1.000000,17.000000,1.150000,2.110000,70.653000,0.000000
75%,12.000000,10.000000,19.000000,1.670000,2.440000,82.481000,0.000000
max,18.000000,16.000000,30.000000,3.060000,2.890000,109.570000,1.000000


In [2]:
final_df.head()

,Team,Group Stage Point,Highest Round,Played,Goal Difference,Points,League Point,Champion
0,1718APOEL,2.0,0.0,15.0,1.53,2.20,21.550,0.0
1,1718Anderlecht,3.0,0.0,21.0,0.24,1.90,38.500,0.0
2,1718Atlético Madrid,7.0,0.0,17.0,1.00,2.12,106.998,0.0
3,1718Barcelona,14.0,8.0,17.0,2.24,2.65,106.998,0.0
4,1718Basel,12.0,16.0,19.0,1.21,2.00,30.200,0.0
